In [2]:
import torch
import torch.nn as nn
from bindsnet.datasets import MNIST
from bindsnet.network import Network
from bindsnet.encoding import poisson_loader
from bindsnet.network.monitors import Monitor
from bindsnet.network.topology import Connection
from bindsnet.network.nodes import LIFNodes, Input

In [3]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        return self.linear(x)

    
network = Network(dt=1.0)
inpt = Input(784, shape=(28,28)); network.add_layer(inpt, name="I")
output = LIFNodes(625, thresh=-52+torch.randn(625)); network.add_layer(output, name="O")
network.add_connection(Connection(inpt, output, w=torch.randn(inpt.n, output.n)), "I", "O")
network.add_connection(Connection(output, output, w=0.5*torch.randn(output.n, output.n)), "O", "O")
network.add_monitor(Monitor(output, ["s"], time=250), name="output_spikes")


images, labels = MNIST("../../data/MNIST").get_train()
loader = zip(poisson_loader(images*0.25, time=250), iter(labels))


training_pairs = []
for i, (datum, label) in enumarate(loader):
    network.run(inpts={"I", datum}, time=250)
    training_pairs.append([network.monitors["output_spikes"].get("s").sum(-1), label])
    network.reset_state_variables()
    
    if (i+1)%50==0:
        print("Train progress: (%d/500)"%(i+1))
    if (i+1)==500:
        print()
        break

correct, total = 0, 0
for s, label in test_pairs:
    output = model(s)
    _, predicted = torch.max(output.data.unsqueeze(0), 1)
    total += 1
    correct += int(predicted == label.long())

print("Accuracy of logistic regression on 500 test examples: %.2f %%¥n"%(100*correct/total))

TypeError: __init__() missing 1 required positional argument: 'root'